In [1]:
%run preamble_scripts.py

# Simulation study: the well specified case 1000 observations

In [2]:
n_sim, nobs, expo = 1000, 1000, 250
err_pnl, popsize = 3e5, 8000

In [3]:
# Setting the true model
f1, f2 = bs.loss_model("Weibull", ["k1", "β1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])

f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)

%time PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

CPU times: user 39.5 s, sys: 24.3 ms, total: 39.5 s
Wall time: 39.5 s


array([-170.08350669, -124.97720512,  -54.78927556])

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [4]:
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
def fit_one(k, f, f_name,  prior, X):
    trace, log_marg, DIC, WAIC = bs.smc(X, f, popsize, prior, verbose = False)
    PnLs = np.array(f.PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
    cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
    return([k, f_name , nobs, log_marg, trace["γ"].mean(), parms_true[-1] ] + cap.tolist() + true_cap.tolist())
def fit_all(k):
    print("simulation #"+ str(k))
    col_names = ["sim", "model_name", "nobs", "log_marg", "γ_map", "γ_true", "cap005", "cap01", "cap05", "cap005_true", "cap01_true", "cap05_true" ]
    df_temp = pd.DataFrame([fit_one(k, fs[i], f_names[i], prior_spliced_model[i], Xs[k]) for i in range(len(fs))], columns = col_names)
    df_temp['posterior_probability'] = np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)) / \
    np.sum(np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)))  
    γ_BMA = np.dot(df_temp['posterior_probability'].values, df_temp['γ_map'])
    cap_bma = np.dot(df_temp['posterior_probability'].values, df_temp[["cap005", "cap01", "cap05"]].values)
    BMA_df = pd.DataFrame([[k, "BMA" , nobs, None, γ_BMA, parms_true[-1] ] + cap_bma.tolist() + true_cap.tolist()],
            columns = col_names)
    df_temp = pd.concat([df_temp, BMA_df])
    return(df_temp)
%time res = Parallel(n_jobs= 50)(delayed(fit_all)(k) for k in range(n_sim))
pd.concat(res).to_csv("../../Data/Simulations/simu_wellspec_"+str(nobs)+".csv", sep=',')

/home/pgoffard/miniconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 2min 36s, sys: 13.6 s, total: 2min 50s
Wall time: 6h 25min 51s


In [5]:
pd.concat(res)

,sim,model_name,nobs,log_marg,γ_map,γ_true,cap005,cap01,cap05,cap005_true,cap01_true,cap05_true,posterior_probability
0,0,Weibull_Log-Logistic,1000,-695.198951,2.387208,1.5,-432.168760,-276.810293,-77.126270,-170.083507,-124.977205,-54.789276,2.137331e-01
1,0,Weibull_Lomax,1000,-694.083278,1.608097,1.5,-215.514123,-144.505447,-44.311241,-170.083507,-124.977205,-54.789276,6.522324e-01
2,0,Weibull_Inverse-Weibull,1000,-695.665582,3.515591,1.5,-434.846582,-281.252643,-91.158412,-170.083507,-124.977205,-54.789276,1.340345e-01
3,0,Inverse-Weibull_Log-Logistic,1000,-740.635753,0.803413,1.5,-651.189465,-409.248294,-124.220259,-170.083507,-124.977205,-54.789276,3.952932e-21
4,0,Inverse-Weibull_Lomax,1000,-738.626819,0.363668,1.5,-279.344012,-192.493223,-65.993288,-170.083507,-124.977205,-54.789276,2.947057e-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,999,Weibull_Inverse-Weibull,1000,-815.384584,2.637527,1.5,-831.543858,-547.638845,-186.008731,-170.083507,-124.977205,-54.789276,2.413740e-01
3,999,Inverse-Weibull_Log-Logistic,1000,-853.287709,0.340186,1.5,-2202.672013,-1293.289220,-385.557676,-170.083507,-124.977205,-54.789276,8.347808e-18
4,999,Inverse-Weibull_Lomax,1000,-847.631221,0.124457,1.5,-754.498888,-504.133562,-188.927577,-170.083507,-124.977205,-54.789276,2.388658e-15
5,999,Inverse-Weibull_Inverse-Weibull,1000,-855.873263,2.150676,1.5,-832.017200,-525.724222,-163.357166,-170.083507,-124.977205,-54.789276,6.290430e-19
